In [1]:
# import dependencies
import pandas as pd 
import requests
import time
import pymongo
from  tqdm import tqdm
from pprint import pprint

#import local libraries
import functions_mongo as mdb
import funtions_webscraping as web

In [2]:
##### Initialize Global Variables #####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#create database
db = client.visualizing_sep

#create collection to hold SEP TOC
toc_collection = db.sep_win2019_toc



In [18]:
# scrape TOC from SEP 

print("Starting: SEP TOC Scrape\n")

#TOC to scrape
search_url = "https://plato.stanford.edu/archives/win2019/contents.html"

#url to generate absolute paths from
base_url = "https://plato.stanford.edu/archives/win2019/"

#current TOC edition
toc_edition = "Winter 2019"

#hold all individual TOC entries 
toc_list = []

#scrape selected webpage and save HTML file
soup = web.scrape_web_page(search_url, "html_files/win2019_toc.html")

#get all links with an HREF attribute in the content page
links_in_toc = soup.find(id="content").findAll("a", href=True)

print(len(links_in_toc))

file_toclist = open("html_files/sep_toc_list.txt","w",encoding="UTF-8")
file_links = open("html_files/sep_toc_links.txt", "w", encoding="UTF-8")

for link in links_in_toc:
    link_text = link.get_text()
    link_href = link["href"]

    if link_href != '#pagetopright':
        #create dictionary to append to mongo
        entry = {"base_url":base_url, "link_url":link_href}
        if entry not in toc_list:
            toc_list.append(entry)
            file_toclist.write(f"{link_text}:{link_href}\n")
            file_links.write(f"{link}\n")

# #insert into Mongo
toc_collection.insert_many(toc_list)


print("Completed: SEP TOC Scrape\n")


Starting: SEP TOC Scrape

html_files/win2019_toc.html
2275
Completed: SEP TOC Scrape



In [19]:
# pages = toc_collection.find({'link_url': {'$regex': '^entries/a'}}).sort('link_url')
pages = toc_collection.find().sort('link_url')
pages_list = list(pages)

for page in tqdm(pages_list, desc="Processing"):
    base_url = page['base_url']
    link_url = page['link_url']
    slash_pos = link_url.rfind("/",0,len(link_url)-1)
    page_name = link_url[slash_pos+1:len(link_url)-1]
    page_url = f"{base_url}{link_url}"
    html_save_as = f"html_files/sep/{page_name}.html"
    try:
        print(html_save_as)
        #soup = web.scrape_web_page(page_url, html_save_as)
    except:
        print(f"Scraped failed @ {page_name}")





l
html_files/sep/nietzsche-life-works.html
html_files/sep/nietzsche-moral-political.html
html_files/sep/nietzsche.html
html_files/sep/nishida-kitaro.html
html_files/sep/nominalism-mathematics.html
html_files/sep/nominalism-metaphysics.html
html_files/sep/nonexistent-objects.html
html_files/sep/nonidentity-problem.html
html_files/sep/nonwellfounded-set-theory.html
html_files/sep/nothingness.html
html_files/sep/novalis.html
html_files/sep/nozick-political.html
html_files/sep/numenius.html
html_files/sep/oakeshott.html
html_files/sep/object.html
html_files/sep/obligationes.html
html_files/sep/occasionalism.html
html_files/sep/ockham.html
html_files/sep/olivi.html
html_files/sep/olympiodorus.html
html_files/sep/omnipotence.html
html_files/sep/omnipresence.html
html_files/sep/omniscience.html
html_files/sep/ontological-arguments.html
html_files/sep/ontological-commitment.html
html_files/sep/operationalism.html
html_files/sep/ordinary-objects.html
html_files/sep/organ-donation.html
html_file